<a href="https://colab.research.google.com/github/kei2/colab/blob/main/FasterWhisper_Youtube_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faster Whisper による Youtube 動画からの文字起こし
参考: https://tt-tsukumochi.com/archives/7701


### Whisper

| モデルサイズ | パラメータ数 | 英語専用 | モデル | 多言語モデル | 必要なVRAM | 相対速度 | <br>
| tiny | 39 M | tiny.en | tiny | ~1GB |	~32x | <br>
| base | 74M | base.en | base | ~1GB | ~16x | <br>
| small | 244M | small.en | small | ~2GB | ~6x | <br>
| medium | 769M | medium.en | medium | ~5GB | ~2x | <br>
| large | 1550 M | なし | large | ~10GB | 1x |


### Faster Whisper

Github：https://github.com/guillaumekln/faster-whisper

Faster-whisper は、OpenAI の Whisper モデルを再実装した高速推論エンジンで、CTranslate2 という Transformer モデルを活用しています。OpenAI の公式モデルを軽量化し、独自の最適化により最大4倍の高速化が実現されています。また、軽いと評判の Whisper.cpp よりも高速に動作し、GPU による高速化の恩恵を受けることができます。本家のwhisperと比較しても高速度で処理することが可能です。

Implementation	Precision	Beam size	Time	Max. GPU memory	Max. CPU memory <br>
openai/whisper	fp16	5	4m30s	11325MB	9439MB <br>
faster-whisper	fp16	5	54s	4755MB	3244MB <br>
faster-whisper	int8	5	59s	3091MB	3117MB <br>

開発元は OpenAI ではなく、あくまで OpenAI のモデルをベースに改良・再実装を行ったものである点に注意してください。Faster-whisperを使用するメリットは以下のようにまとめることができます。

- 外部に音声データを送信したくない場合
- 25MB 以上の音声データを送信したい場合
- 無料で利用したい場合
- 他の処理と組み合わせて使用したい場合(例えばYouTube動画をダウンロードしてから文字起こしするなど)

### yt-dlp

Github：https://github.com/yt-dlp/yt-dlp

yt-dlpは、YouTubeをはじめとするオンライン動画サイトから動画や音声をダウンロードするための便利なツールです。かつては「youtube-dl」という同様のツールが人気を博していましたが、近年は開発が停滞し、ダウンロード速度の低下が問題となっていました。そこで登場したのが、youtube-dlの改良版とも言えるyt-dlpです。

yt-dlpの最大の特徴は、youtube-dlと比較してダウンロード速度が格段に速いことです。この速度向上により、ユーザーは手間なく迅速に動画や音声ファイルを入手することができます。また、動画だけでなく、音声ファイルのみを抽出することも可能であり、用途に合わせて選択できる柔軟性も提供しています。

---


In [2]:
!pip install faster-whisper
!python -m pip install -U yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 120.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 日本語の文字起こし

In [3]:
!yt-dlp -x --audio-format mp3 https://www.youtube.com/watch?v=wK0lHdH70Kw -o audio.mp3

[youtube] Extracting URL: https://www.youtube.com/watch?v=wK0lHdH70Kw
[youtube] wK0lHdH70Kw: Downloading webpage
[youtube] wK0lHdH70Kw: Downloading ios player API JSON
[youtube] wK0lHdH70Kw: Downloading android player API JSON
[youtube] wK0lHdH70Kw: Downloading m3u8 information
[info] wK0lHdH70Kw: Downloading 1 format(s): 251
[download] Destination: audio.webm
[download] 100% of   14.21MiB in 00:00:00 at 33.93MiB/s
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.webm (pass -k to keep)


In [4]:
from faster_whisper import WhisperModel

model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe("audio.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'ja' with probability 0.998535
[0.00s -> 4.32s] 首相がやる気になればやれますよね だから菅さん降ろされるんだから
[4.32s -> 12.80s] 菅さんは総務省出身だから知ってるはずなんだよ でもこれプラチナバンド渡さないとラクタンモバイル終わりますよね
[12.80s -> 22.10s] もう言ってもいいと思うけど三木谷さんとこに話持ってたでしょ 本人は途中までちょっと塗り気味のとこあったんだけどそれが途中で
[22.10s -> 26.80s] 総務省で携帯電話用のプラチナバンドの再割当てが今議論されています
[26.80s -> 32.38s] iPhoneとかに入っている通信用のチップって600MHzぐらいから対応してるんですよね
[32.38s -> 38.92s] 600MHz単位のバンドと700MHz単位のバンド この部分が地デジに割り当てられてるんですけど
[38.92s -> 46.50s] 地デジっていうのはいくらテレビ局が多い地域でも10局ないんですよね つまり10チャンネル未満でいいわけですよ
[46.50s -> 56.08s] でも地デジのバンドってね13チャンネルから62チャンネルぐらいまでありますよね だからそこに何か蒸し食い的に電波がなぜか割り当てられてて
[56.08s -> 61.14s] あれも謎議論ですよね 40チャンネルのうち8チャンネルぐらいしか使ってない
[61.14s -> 68.78s] だから30チャンネル以上空いてるんですよ そこは整理したらそこ全部携帯に使えんじゃんって話
[68.78s -> 73.10s] だからこれがわかりやすい これはホワイトスペースっていうふうに言うんですけど
[73.10s -> 78.78s] 今の割り当てで各地バラバラに電波を 赤いとこね割り当ててるんで
[78.78s -> 84.70s] 一つのまとまった地域として端末が使えないんですよ 技術的にやろうと思うと
[84.70s -> 91.32s] ダイナミックに周波数変更すればできるんだけど そのテレビ局が嫌がるから全部各整理しちゃって
[91.32s -> 100.52s] テレビの電波を例えば13から19チャンネルにまとめて 20



---
### 中国語の文字起こし


In [5]:
!yt-dlp -x --audio-format mp3 https://www.youtube.com/watch?v=M6-a-QLlNfw -o audio2.mp3

[youtube] Extracting URL: https://www.youtube.com/watch?v=M6-a-QLlNfw
[youtube] M6-a-QLlNfw: Downloading webpage
[youtube] M6-a-QLlNfw: Downloading ios player API JSON
[youtube] M6-a-QLlNfw: Downloading android player API JSON
[youtube] M6-a-QLlNfw: Downloading m3u8 information
[info] M6-a-QLlNfw: Downloading 1 format(s): 251
[download] Destination: audio2.webm
[download] 100% of   25.39MiB in 00:00:00 at 35.00MiB/s
[ExtractAudio] Destination: audio2.mp3
Deleting original file audio2.webm (pass -k to keep)


In [6]:
from faster_whisper import WhisperModel

model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe("audio2.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'zh' with probability 0.991699
[0.00s -> 2.00s] 最近压力好大
[3.00s -> 4.00s] 你怎么了
[4.00s -> 6.00s] 是工作不顺利吗
[7.00s -> 8.00s] 工作还好
[8.00s -> 11.00s] 就是家里老催着我相亲
[12.00s -> 13.00s] 你才多大
[13.00s -> 15.00s] 有什么可着急的
[16.00s -> 17.00s] 我也觉得
[17.00s -> 19.00s] 而且我根本不想结婚
[20.00s -> 22.00s] 现在的生活成本太高了
[22.00s -> 24.00s] 年轻人都不愿意结婚
[25.00s -> 26.00s] 那你怎么办
[26.00s -> 27.00s] 我怎么办
[27.00s -> 28.00s] 陈忆
[28.00s -> 29.00s] 给你一些好消息
[30.00s -> 31.00s] 弄不好
[31.00s -> 32.00s] 要去偷
[32.00s -> 33.00s] 这里
[33.00s -> 36.00s] 她已经买 fromsteam只不过有九户户了
[36.00s -> 37.00s] 好
[37.00s -> 38.00s] 我觉得最好是
[38.00s -> 41.00s] 我会收到
[41.00s -> 42.00s] 但是
[42.00s -> 43.00s] 这件事情
[44.00s -> 45.00s] 我会
[45.00s -> 46.00s] 了解
[46.00s -> 47.00s] 自己的秘密
[47.00s -> 48.00s] 请你
[48.00s -> 49.22s] 听我
[49.22s -> 50.00s] 道歉
[50.00s -> 54.00s] 工作还好,就是家里老催着我相亲
[55.00s -> 58.00s] 你才多大,有什么可着急的
[59.00s -> 62.00s] 我也觉得,而且我根本不想结婚
[63.00s -> 67.00s] 现在的生活成本太高了,年轻人都不愿意结婚
[69.00s -> 72.00s] 而且我一直觉得一个人挺自由的
[73.00s -> 76.00s] 是啊,你自己觉得开心就行

### 中国語の歌の文字起こし

In [7]:
!yt-dlp -x --audio-format mp3 https://www.youtube.com/watch?v=0lu8nEabNEE -o audio3.mp3

[youtube] Extracting URL: https://www.youtube.com/watch?v=0lu8nEabNEE
[youtube] 0lu8nEabNEE: Downloading webpage
[youtube] 0lu8nEabNEE: Downloading ios player API JSON
[youtube] 0lu8nEabNEE: Downloading android player API JSON
[youtube] 0lu8nEabNEE: Downloading player f980f2a9
[youtube] 0lu8nEabNEE: Downloading m3u8 information
[info] 0lu8nEabNEE: Downloading 1 format(s): 251
[download] Destination: audio3.webm
[download] 100% of    2.32MiB in 00:00:00 at 11.27MiB/s
[ExtractAudio] Destination: audio3.mp3
Deleting original file audio3.webm (pass -k to keep)


In [8]:
from faster_whisper import WhisperModel

model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe("audio3.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

Detected language 'zh' with probability 0.994141
[0.00s -> 4.00s] 幸福拍手歌
[30.00s -> 34.00s] 如果感到幸福你就多多叫
[34.00s -> 37.00s] 如果感到幸福就快快多多叫
[37.00s -> 41.00s] 看那大家都一起多多叫
[41.00s -> 45.00s] 如果感到幸福你就拍拍肩
[45.00s -> 48.00s] 如果感到幸福你就拍拍肩
[48.00s -> 52.00s] 如果感到幸福就快快拍拍肩
[52.00s -> 56.00s] 看那大家都一起拍拍肩
[56.00s -> 59.00s] 如果感到幸福你就閃閃耀
[59.00s -> 62.00s] 如果感到幸福你就閃閃耀
[62.00s -> 66.00s] 如果感到幸福就快快閃閃耀
[66.00s -> 69.00s] 看那大家都一起閃閃耀
[69.00s -> 73.00s] 如果感到幸福你就擠擠牙
[73.00s -> 77.00s] 如果感到幸福你就擠擠牙
[77.00s -> 81.00s] 如果感到幸福就快快擠擠牙
[81.00s -> 84.00s] 看那大家都一起擠擠牙
[89.00s -> 93.00s] 如果感到幸福你就拍拍手
[93.00s -> 97.00s] 如果感到幸福你就拍拍手
[97.00s -> 101.00s] 如果感到幸福就快快拍拍手
[101.00s -> 105.00s] 看那大家都一起拍拍手
[120.00s -> 123.00s] 如果感到幸福你就拍拍手
[123.00s -> 127.00s] 如果感到幸福你就拍拍手
[127.00s -> 131.00s] 如果感到幸福就快快拍拍手
[131.00s -> 134.00s] 看那大家都一起拍拍手
[149.00s -> 152.00s] 如果感到幸福你就擠擠牙
[152.00s -> 155.00s] 如果感到幸福你就擠擠牙
[155.00s -> 158.00s] 如果感到幸福你就擠擠牙
[158.00s -> 161.00s] 如果感到幸福你就擠擠牙
[161.00s -> 164.00s] 如果感到幸福你就擠擠牙
[164.00s -> 167.00s] 如果感到幸福你就擠擠牙
[167.0



---

中国語 ⇒ 日本語 翻訳もやってみる
(なぜか中国語⇒英語になってしまった)


In [9]:
!pip install transformers sentencepiece sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=d9bfe2fd6c4595bd613abd242244421913aff4c8fbbd7dee6af77779970b8339
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [22]:
from transformers import pipeline
cj_translator = pipeline('translation', model="ssmisya/zh-jp_translator")
#cj_translator = pipeline('translation', model='K024/mt5-zh-ja-en-trimmed')
#cj_translator = pipeline('translation', model="Helsinki-NLP/opus-mt-tc-big-zh-ja") // モデルが上手くできていない様だ


In [23]:
cj_translator("生日快乐.")

[{'translation_text': '生日快乐'}]

In [24]:
from faster_whisper import WhisperModel

model_size = "large-v2"

model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe("audio2.mp3", beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    print(cj_translator(segment.text))

Detected language 'zh' with probability 0.991699
[0.00s -> 2.00s] 最近压力好大
[{'translation_text': 'I recentsimal'}]
[3.00s -> 4.00s] 你怎么了
[{'translation_text': 'Why Are You?'}]
[4.00s -> 6.00s] 是工作不顺利吗
[{'translation_text': 'Is it not working well?'}]
[7.00s -> 8.00s] 工作还好
[{'translation_text': 'Working well.'}]
[8.00s -> 11.00s] 就是家里老催着我相亲
[{'translation_text': '这就是家里的人们 一直叫我去爱上我'}]
[12.00s -> 13.00s] 你才多大
[{'translation_text': "Because you're the only person who's big."}]
[13.00s -> 15.00s] 有什么可着急的
[{'translation_text': 'Something is going to be a little bit anxious.'}]
[16.00s -> 17.00s] 我也觉得
[{'translation_text': "I think I'm going to be a little bit more like this."}]
[17.00s -> 19.00s] 而且我根本不想结婚
[{'translation_text': "And I don't want to marry anymore."}]
[20.00s -> 22.00s] 现在的生活成本太高了
[{'translation_text': 'These days, living costs are too high.'}]
[22.00s -> 24.00s] 年轻人都不愿意结婚
[{'translation_text': 'Teenagers are unwilling to marry.'}]
[25.00s -> 26.00s] 那你怎么办
[{'translation_text': 